In [1]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

from scipy.optimize import minimize
from scipy.optimize import differential_evolution    

from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.insert(1,'../covid-website')

import SISV as s
from data import Data


In [5]:

#----------------------------------------------------------
params = {
    "population"    : 1e6,
    "i0"            : 1,
    "death_rate"    : 0.5*1e-2,
    "detection_rate": 10e-2,

    'exp_stages'  : 3,
    'inf_stages'  : 3,
    'crit_stages' : 3,
    'test_stages'  : 3,
    'death_rate'  : 0.73e-2,
    'gamma_exp'   : 1/4,
    'gamma'       : 1/4,
    'gamma_crit'  : 1/(17+21),
    'gamma_pos'   : 1/(5+10),
    
    "immun"         : 0,
    "vacc_start"    : 365,
    "vacc_rate"     : 0.3/365,  #30% of population per year
    "vacc_immun"    : 1/180,
    
    "interv"         : 'piecewise constant',
    "init_beta"     : 'const',     
    'segments'      : 0,
    "beta0"          : 1.4/4,
}

n=3000
x = np.arange(n)

y = s.SISV(x, params)

p = figure(plot_width=800, plot_height=600 )#, y_axis_type="log")
p.line(x, y[:,s.cF], line_width=3, line_color='green', line_dash='solid', alpha=0.3)

show(p)



In [6]:
#--------------------------------------------------------------
#various log-likelihood functions that can be used by fit_model()
def loglik_leastsquarerel(y, yhat, alpha):
    #y[] are observed values; yhat are model prediction
    leastsquare = np.nansum( (np.log(yhat)-np.log(y))**2  )  
    return leastsquare

def loglik_leastsquare(y, yhat, alpha):
    #y[] are observed values; yhat are model prediction
    leastsquare = np.nansum( (yhat-y)**2  )  
    return leastsquare

def loglik_poisson(y, yhat, alpha):
    #y[] are observed values; yhat are model prediction
    negLL = -np.nansum( -yhat + y * np.log(yhat)  )  #removed constant terms for minimization
    return negLL

def loglik_negbin(y, yhat, alpha):
    #y[] are observed values; yhat are model prediction
    r = 1/alpha
    #negLL = -np.nansum( loggamma(y+r) -loggamma(y+1) - loggamma(r) + y*np.log(yhat) + r*np.log(r) - (y+r)*np.log(yhat+r) )
    negLL = -np.nansum( y*np.log(yhat)  - (y+r)*np.log(yhat+r) )  #removed constant terms to speed minimization <more sensitive to initial guess ???
    return negLL       

#------------------------------
#estimate alpha for negative binomial
#https://dius.com.au/2017/08/03/using-statsmodels-glms-to-model-beverage-consumption/
def calc_alpha(data,fit):
    var = (np.power(data-fit,2) - data)/fit
    X = fit[:,np.newaxis]
    ols = LinearRegression(fit_intercept=False).fit(np.nan_to_num(X), np.nan_to_num(var))
    alpha = ols.coef_[0]
    return alpha

#--------------------------------------------------------------
#fit the given model using the given maximum likelihood
def fit_model(x, y, model_func, constants, loglik_func, guess, bounds, alpha=1):
   
    #this function is called by the scipy's minimize()
    #it returns the negative log likelihood of the model prediction given the model parameters (optimization target) and the constants
    #it is closure to access calibration data x (to compute the prediction) and y (to compute the likelihood)
    def regression_func(params, constants):
        #make a prediction using the given model params
        yhat = model_func(x, params, constants)
        # compute negative of likelihood 
        negLL = loglik_func(y, yhat, alpha)
        return negLL    
    
    mle = minimize(regression_func, x0=guess, bounds=bounds, args=constants, method="L-BFGS-B")#, method ="Nelder-Mead")
    #display(mle)
    
    res = model_func(x, mle.x, constants)
    return mle.x, mle.fun, res    #mle.x is the array of calibrated model parameters; mle.fun is the loglikelihood (without constant terms); res is the model forecast for input range


In [7]:
source = 'Johns Hopkins'    
region = 'US'
state = 'California'

cutoff_positive = 1
cutoff_death = 1
truncate = 0
window = 14

d = Data(source=source, region=region, state=state, county="", cutoff_positive=cutoff_positive, cutoff_death=cutoff_death, truncate=truncate) 


In [8]:
%%time
def SISV_fit_model(x,y):
    
    def merge_params(params, constants):
        param_list = ['i0', 'beta0']
        p = constants.copy()
        for i, param in enumerate(param_list):
            p[param] = params[i]
            
        n = constants['segments']
        for i in range(1,n+1):
            p['beta{}'.format(i)] = params[len(param_list)+i-1]

        return p
    
    def fit_model_inner(x, params, constants):
        p = merge_params(params, constants)
        yhat = s.SISV(x, p)
        return yhat[:,s.cF] - y
    
    
    gamma = 1/4

    constants = {
    "population"    : 20e6,
    "death_rate"    : 0.5*1e-2,
    "detection_rate": 10e-2,
    "gamma"         : gamma,
    "gamma_crit"    : 1/14,
    "gamma_pos"     : 1/5,
    "immun"         : 0,
    "vacc_start"    : 365,
    "vacc_rate"     : 0/365,  #30% of population per year
    "vacc_immun"    : 1/180,
    
    "interv"         : 'piecewise linear',
    "init_beta"     : 'const',     
    'segments'      : 4,       
    't1'            : 37,
    't2'            : 45,
    't3'            : 286,
    't4'            : 294,
    }

    
    guess = (32000, 3*gamma, 0.3*gamma, 1.1*gamma, 1.8*gamma, 1.7*gamma)
    
    bounds  = [
        (1, 100e3),                 #i0
        (0.1*gamma, 10*gamma),      #beta0
        (0.1*gamma, 10*gamma),      #beta1
        (0.1*gamma, 10*gamma),
        (0.1*gamma, 10*gamma),
        (0.1*gamma, 10*gamma),
    ]

    p, fun, res = fit_model(x, y, model_func=fit_model_inner, loglik_func=loglik_leastsquare, constants=constants, guess=guess, bounds=bounds, alpha=1)
    
    params = merge_params(p, constants)
    yy = s.SISV(x, params)
    
    return yy, params

#y_fit_model, params_fit_model = SISV_fit_model(d.x[d.minD:], d.fatalities[d.minD:])


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.44 µs


In [9]:
#transform auxiliary values (which are bounded [0,1] into the breakpoint dates)
def aux_to_breakpoints(aux, x0, xn, minwindow):
    breakpoints = []
    bi1 = x0
    n=len(aux)
    for i,a in enumerate(aux):
        bi = a * (xn-(n-i)*minwindow - (bi1+minwindow)) + bi1+minwindow
        breakpoints.append(bi)
        bi1=bi   
    return breakpoints

#aux_to_breakpoints([0.5, 0.5], 0, 100, 10)

In [10]:
%%time
def SISV_diffevol2(x, y):
    
    minwindow=7
    

    
    def merge_params_1(params, constants):
        p = constants.copy()
        breakpoints = aux_to_breakpoints(params, x[0], x[-1], minwindow)
        n = p['segments']
        for i in range(1,n+1):
            p['t{}'.format(i)] = breakpoints[i-1]

        #print(p)
        return p
    
    def merge_params_2(params, constants):
        p = constants.copy()
        param_list = ['i0', 'beta0']
        for i, param in enumerate(param_list):
            p[param] = params[i]

        n = p['segments']
        for i in range(1,n+1):
            p['beta{}'.format(i)] = params[len(param_list)+i-1]

        return p
    
    def fit_model_inner(x, params, constants):
        p = merge_params_2(params, constants)
        yhat = s.SISV(x, p)
        return yhat[:,s.cF] - y
    
    
    def diffevol_inner(params, constants):
        p = merge_params_1(params, constants)
        #yhat = s.SISV(x, p)
        #negLL = loglik_leastsquare(y, yhat[:, s.cF], alpha=0)
        p, fun, res = fit_model(x, y, model_func=fit_model_inner, loglik_func=loglik_leastsquare, constants=p, guess=guess_1, bounds=bounds_1, alpha=1)
        return fun

    gamma = 1/6

    constants = {
    "population"    : 20e6,
    "death_rate"    : 0.5*1e-2,
    "detection_rate": 10e-2,
    "gamma"         : gamma,
    "gamma_crit"    : 1/14,
    "gamma_pos"     : 1/5,
    "immun"         : 0,
    "vacc_start"    : 365,
    "vacc_rate"     : 0/365,  #30% of population per year
    "vacc_immun"    : 1/180,
    
    "interv"         : 'piecewise constant',
    "init_beta"     : 'const',     
    'segments'      : 3,
    }
    
    n = constants['segments']

    guess_1 = [1000, 2*gamma]
    
    bounds_1  = [
        (1, 100e3),                 #i0
        (0.1*gamma, 5*gamma),      #beta0
    ]

    bounds = []
    
    for i in range(n):
        guess_1.append(1*gamma)
        bounds_1.append((0.1*gamma, 5*gamma))
        bounds.append((0,1))
    
    fit = differential_evolution(func=diffevol_inner, bounds=bounds, args=[constants])
    
    p = merge_params_1(fit.x, constants)

    pp, fun, res = fit_model(x, y, model_func=fit_model_inner, loglik_func=loglik_leastsquare, constants=p, guess=guess_1, bounds=bounds_1, alpha=1)
    
    ppp = merge_params_2(pp, p)
    
    print(ppp)
    yy = s.SISV(x, ppp)
    
    return yy, ppp
    
    

#y_diffevol_2, params_diffevol_2 = SISV_diffevol2(d.x[d.minD:], d.fatalities[d.minD:])



CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


In [11]:
%%time
def SISV_diffevol1(x, y):
    
    minwindow=7
    def merge_params(params, constants):
        p = constants.copy()

        param_list = ['i0', 'beta0']
        for i, param in enumerate(param_list):
            p[param] = params[i]
        
        n = p['segments']
        breakpoints = aux_to_breakpoints(params[-n:], x[0], x[-1], minwindow)
        
        for i in range(1,n+1):
            p['beta{}'.format(i)] = params[len(param_list)+i-1]
            p['t{}'.format(i)] = breakpoints[i-1]

        return p
    
    def diffevol_inner(params, constants):
        p = merge_params(params, constants)
        yhat = s.SISV(x, p)
        negLL = loglik_leastsquare(y, yhat[:, s.cF], alpha=0)
        return negLL

    gamma = 1/4

    constants = {
    "population"    : d.population, #20e6,
    "death_rate"    : 0.5*1e-2,
    "detection_rate": 10e-2,
    "gamma"         : gamma,
    "gamma_crit"    : 1/14,
    "gamma_pos"     : 1/14,
    "immun"         : 0,
    "vacc_start"    : 365,
    "vacc_rate"     : 0/365,  #30% of population per year
    "vacc_immun"    : 1/180,
    
    "interv"         : 'piecewise constant',
    "init_beta"     : '',     
    "segments"      : 4,
    }
    
    
    n = constants['segments']

    bounds  = [
        (1, 100e3),                 #i0
        (0.1*gamma, 5*gamma),      #beta0
    ]

    for i in range(n):
        bounds.append((0.1*gamma, 5*gamma))  #betai
    
    for i in range(n): #auxiliary variables to solve for time breakpoints ti
        bounds.append((0,1))
    
    fit = differential_evolution(func=diffevol_inner, bounds=bounds, args=[constants])
    
    p = merge_params(fit.x, constants)
    yy = s.SISV(x, p)
    return yy, p
    

#y_diffevol_1, params_diffevol_1 = SISV_diffevol1(d.x[d.minD:], d.fatalities[d.minD:])





CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [12]:
%%time

from lmfit import Minimizer, Parameters, report_fit
#---------------------------------------------------
#Piece-wise linear solves times and beta for a given number of segments 
#---------------------------------------------------

def SISV_lmfit_1(x, y, overrides={}, solver=''):
#def calibrate_positives_piecewiselinear_multiple(d, label, segments, window, startx, overrides={}):


    def merge_params(params, constants):
        p = params.valuesdict()  #params should be a LMFIT Parameters instance; constants should be a dict
        for i, (k,v) in enumerate(constants.items()):
            if k not in p:  #do not override values that may already be in the Parameters array
                p[k]=v
        return p

    def lmfit_inner(params, x, constants, data=None):
        p = merge_params(params, constants)
        yhat = s.SISV(x, p)
        if data is None:
            return yhat
        else:
            return yhat[:,s.cF] - data
        #negLL = loglik_leastsquare(y, yhat[:, s.cF], alpha=0)
        #return negLL


    gamma = 1/4
    segments = 4
    window = 7

    tmin = x[0]
    tmax = x[-1]

    params = Parameters()
    #(name, value, vary, min, max, expr)
    params.add_many( 
                     ('gamma',          gamma, False),
                     ('gamma_infec',    1/7, False),
                     ('gamma_pos',      1/14, False),
                     ('gamma_crit',     1/14, False),
                     ('death_rate',     0.5e-2, False),
                     ('detection_rate', 5e-2, False),

                     ('population',     d.population, False),
                     ('f0',             y[0], False),

        
                     ('immun',          0, False),
                     ('vacc_start',     365, False),
                     ('vacc_rate',      0, False),
                     ('vacc_immun',     1/180, False),

        
                     ('segments',       segments         , False),

                     ('i0',             100        , True, 1, 1e6),
                     ('beta0',          2*gamma       , True, 0.1*gamma, 5*gamma),                 
                   )

    params.add('t0',value=tmin, vary=False)
    for i in range(1, segments+1):  
        ti   = tmax - (segments-i) * window
        ti1  = 't{}'.format(i-1)
        auxi = 'aux{}'.format(i-1)

        params.add(auxi,value=0.1, vary=True, min=0, max=1)
        params.add('t{}'.format(i), expr="{auxi}*({ti}-{ti1}-{window})+{ti1}+{window}".format(auxi=auxi, ti=ti, ti1=ti1, window=window), min=0, max=500)

        params.add('beta{}'.format(i), value= 1*gamma, vary=True,  min=0.1*gamma, max=5*gamma)
        #print(ti)


    #lmfit Parameters cannot accept string values so they get passed in a separate argument
    constants = { 
        'interv':'piecewise constant',
        'init_beta':'const',
    }

    for idx, (k,v) in enumerate(overrides.items()):
        if k in params:
            params[k].set(value=v)
        else:
            constants[k]=v
            
    fitter = Minimizer(lmfit_inner, params, fcn_args=(x, constants, y))
    #result = fitter.minimize()
    #result = fitter.minimize(method='differential_evolution')
    result = fitter.minimize(method=solver)

    #result.params.pretty_print()
    #report_fit(result)
    
    p = merge_params(result.params, constants)
    
    #print(p)
    yy = s.SISV(x, p)
    return yy, p
    
#y_lmfit_1, params_lmfit_1 = SISV_lmfit(d.x[d.minD:], d.fatalities[d.minD:], solver='differential_evolution')


CPU times: user 145 ms, sys: 8.1 ms, total: 153 ms
Wall time: 153 ms


In [13]:
%%time

from lmfit import Minimizer, Parameters, report_fit
#---------------------------------------------------
#Piece-wise linear solves times and beta for a given number of segments 
#---------------------------------------------------

def SISV_lmfit(x, y, overrides={}, solver=''):
#def calibrate_positives_piecewiselinear_multiple(d, label, segments, window, startx, overrides={}):


    def merge_params(params, constants):
        p = params.valuesdict()  #params should be a LMFIT Parameters instance; constants should be a dict
        for i, (k,v) in enumerate(constants.items()):
            if k not in p:  #do not override values that may already be in the Parameters array
                p[k]=v

        n = p['segments']
        
        aux = []
        for i in range(1, n+1):
            aux.append(params['aux{}'.format(i-1)])
            
        breakpoints = aux_to_breakpoints(aux, x[0], x[-1], minwindow)
        
        for i in range(1,n+1):
            #p['beta{}'.format(i)] = params[len(param_list)+i-1]
            p['t{}'.format(i)] = breakpoints[i-1]
                
        return p

    def lmfit_inner(params, x, constants, data=None):
        p = merge_params(params, constants)
        
            
        yhat = s.SISV(x, p)
        if data is None:
            return yhat
        else:
            return yhat[:,s.cF] - data
        #negLL = loglik_leastsquare(y, yhat[:, s.cF], alpha=0)
        #return negLL


    gamma = 1/3
    segments = 7
    minwindow= 7

    tmin = x[0]
    tmax = x[-1]

    params = Parameters()
    #(name, value, vary, min, max, expr)
    params.add_many( 
                     ('gamma',          gamma, False),
                     ('gamma_infec',    gamma, False),
                     ('gamma_pos',      1/14, False),
                     ('gamma_crit',     1/14, False),
                     ('death_rate',     0.5e-2, False),
                     ('detection_rate', 5e-2, False),

                     ('population',     d.population, False),
                     ('f0',             y[0], False),

        
                     ('immun',          0, False),
                     ('vacc_start',     365, False),
                     ('vacc_rate',      0, False),
                     ('vacc_immun',     1/180, False),

        
                     ('segments',       segments         , False),

                     ('i0',             10        , True, 1, 1e4),
                     ('beta0',          2*gamma       , True, 0.1*gamma, 5*gamma),                 
                   )

    for i in range(1, segments+1):  
        auxi = 'aux{}'.format(i-1)
        params.add(auxi,value=0.1, vary=True, min=0, max=1)
        params.add('beta{}'.format(i), value= 1*gamma, vary=True,  min=0.1*gamma, max=5*gamma)

    #lmfit Parameters cannot accept string values so they get passed in a separate argument
    constants = { 
        'interv':'piecewise linear',
        'init_beta':'',
    }

    for idx, (k,v) in enumerate(overrides.items()):
        if k in params:
            params[k].set(value=v)
        else:
            constants[k]=v
            
    fitter = Minimizer(lmfit_inner, params, fcn_args=(x, constants, y))
    #result = fitter.minimize()
    #result = fitter.minimize(method='differential_evolution')
    result = fitter.minimize(method=solver)

    #result.params.pretty_print()
    #report_fit(result)
    
    p = merge_params(result.params, constants)
    
    print(p)
    yy = s.SISV(x, p)
    return yy, p
    
y_lmfit_leastsq, params_lmfit_leastsq = SISV_lmfit(d.x[d.minD:], d.fatalities[d.minD:], solver='leastsq')




KeyError: 'exp_stages'

In [14]:
%%time
#y_lmfit_ampgo, params_lmfit_ampgo = SISV_lmfit(d.x[d.minD:], d.fatalities[d.minD:], solver='ampgo')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [55]:
%%time
y_lmfit_diffevol, params_lmfit_diffevol = SISV_lmfit(d.x[d.minD:], d.fatalities[d.minD:], solver='differential_evolution')

OrderedDict([('gamma', 0.3333333333333333), ('gamma_infec', 0.3333333333333333), ('gamma_pos', 0.07142857142857142), ('gamma_crit', 0.07142857142857142), ('death_rate', 0.005), ('detection_rate', 0.05), ('population', 39512223.0), ('f0', 2), ('immun', 0), ('vacc_start', 365), ('vacc_rate', 0), ('vacc_immun', 0.005555555555555556), ('segments', 7), ('i0', 9867.453987589264), ('beta0', 1.1338973661192366), ('aux0', 0.8892239604235397), ('beta1', 1.1346228081079333), ('aux1', 0.6797916561952367), ('beta2', 1.2330417804196419), ('aux2', 0.8901806381659336), ('beta3', 1.659101690233917), ('aux3', 0.5449682602230242), ('beta4', 1.6608674951415336), ('aux4', 0.5618892197187769), ('beta5', 1.1705664913416747), ('aux5', 0.9202799428449124), ('beta6', 1.664752566608031), ('aux6', 0.8601174046532913), ('beta7', 1.5982984648048075), ('interv', 'piecewise linear'), ('init_beta', ''), ('t1', 286.644349512238), ('t2', 313.60007577600226), ('t3', 328.967706320432), ('t4', 336.5302736110254), ('t5', 34

In [56]:
from bokeh.models import DatetimeTickFormatter, MonthsTicker, NumeralTickFormatter, Legend

def print_params(title, params):
    print('----------')
    print(title)
    print("i0:", params["i0"])
    print("R0:", params['beta0']/params['gamma'])
    for i in range(1, 1+params['segments']):
          print(params['t{}'.format(i)],":", params['beta{}'.format(i)]/params['gamma'])

#print_params("fit_model", params_fit_model)    
#print_params("diffevol_1", params_diffevol_1)
#print_params("diffevol_2", params_diffevol_2)
#print_params("lmfit_1", params_lmfit_1)
print_params("lmfit_leastsq", params_lmfit_leastsq)
#print_params("lmfit_ampgo", params_lmfit_ampgo)
print_params("lmfit_diffevol", params_lmfit_diffevol)
    
p = figure(title='Excess Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")

r0 = p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD+1:], d.dfatalities, size=5, color="red", alpha=0.3)

#plot 7-day rolling average
#rolling = pd.DataFrame(data = d.dfatalities).interpolate().rolling(7).mean()
#r2 = p.line(d.xd[d.minD+1:], rolling.loc[:,0].values, line_width=1, line_color='red')

#r3_fit_model   = p.line(d.xd[d.minD+1:], np.diff(y_fit_model[:,s.cF]), line_width=1, line_color='blue', line_dash='solid', alpha=1)

#r3_diffevol_1 = p.line(d.xd[d.minD+1:], np.diff(y_diffevol_1[:,s.cF]), line_width=1, line_color='black', line_dash='dashed', alpha=1)

#r3_diffevol_2 = p.line(d.xd[d.minD+1:], np.diff(y_diffevol_2[:,s.cF]), line_width=1, line_color='black', line_dash='solid', alpha=1)

#r3_lmfit_1 = p.line(d.xd[d.minD+1:], np.diff(y_lmfit_1[:,s.cF]), line_width=1, line_color='red', line_dash='solid', alpha=1)

r3_lmfit_leastsq = p.line(d.xd[d.minD+1:], np.diff(y_lmfit_leastsq[:,s.cF]), line_width=1, line_color='black', line_dash='solid', alpha=1)

#r3_lmfit_ampgo = p.line(d.xd[d.minD+1:], np.diff(y_lmfit_ampgo[:,s.cF]), line_width=1, line_color='green', line_dash='solid', alpha=1)

r3_lmfit_diffevol = p.line(d.xd[d.minD+1:], np.diff(y_lmfit_diffevol[:,s.cF]), line_width=1, line_color='orange', line_dash='solid', alpha=1)


legend = [
    ("COVID fatalities"   , [r0, r1]),
#    ("COVID 7-day average"   , [r2]),
    
#    ("fit_model", [r3_fit_model]),
#    ("diffevol_1", [r3_diffevol_1]),
#    ("diffevol_2", [r3_diffevol_2]),
#    ("lmfit_1", [r3_lmfit_1]),
    ("lmfit_leastsq", [r3_lmfit_leastsq]),
#    ("lmfit_ampgo", [r3_lmfit_ampgo]),
#    ("lmfit_diffevol", [r3_lmfit_diffevol]),
]

    
p.add_layout(Legend(items=legend, location='center'), 'right')
   
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))

p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker

p.xaxis.major_label_orientation = math.pi/4

p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

#p.toolbar_location = None
show(p)

----------
lmfit_leastsq
i0: 291.9227878916521
R0: 2.928845528361346
61.12308559283373 : 1.1787848377207328
81.99929692503919 : 0.9628241999222529
130.559769089227 : 1.029668206006463
167.65000404701613 : 1.158642429035945
182.99977815977553 : 1.0280784485393144
272.7318734357166 : 1.0357750488902542
279.7318734357166 : 1.2771423644281945
----------
lmfit_diffevol
i0: 9867.453987589264
R0: 3.40169209835771
286.644349512238 : 3.4038684243237998
313.60007577600226 : 3.6991253412589256
328.967706320432 : 4.977305070701751
336.5302736110254 : 4.982602485424601
343.79420780520763 : 3.5116994740250242
350.9835942344691 : 4.994257699824093
357.9977051189389 : 4.794895394414423
